In [1]:
%load_ext autoreload
%autoreload 2

In [88]:
import glob
import os
import json
import random
import string
# 指定したフォルダ内のファイルを検索
cluster_id=3
path_list = glob.glob(f"../data/categorized/{cluster_id}/*.jsonl")


34294

In [95]:
from src.RapidFuzzDedup import dedup_lines,dedup_dir
dedup_dir(0)

1887
1398


In [81]:
len(all_lines),len(deduped_lines)

(37614, 808)

In [83]:
set(check_lines)-set(deduped_lines)

{'(コントローラ上のプロジェクトが改ざんされた可能性がある場合、早期発見することで被害拡大を防ぐことができます。)',
 '(再現性のためには)保存する必要が理解いただけるかと思います。',
 '<当店では入荷時に基本的なテストを行っておりますので、「届いたら壊れていた(怒)」ということは基本的にありません。',
 'AMQPの仕様は、長年にわたって業界ができなかった(あるいは、可能にすることを望まなかった)ことの実現を目指すものだ。',
 'AWSの驚異的なスピードのバージョンアップの恩恵を受けやすくするため、2012年から現在に至るまで、できる限りAWSを活用する取り組みをしています。',
 'Ansibleは、冪等性が担保されていると良く言われていますが、これもモジュールの作り次第です。',
 'Appleが近く発表するかもしれないと噂になっている、新しいタブレット型コンテンツビューアの名称が「iSlate」になる可能性があるようです。',
 'Appleとしては、スタンドアロンアプリにすることで、サービスを促進し、広告主の増加に期待しているようですが、まだこのテストは初期段階にあるために「iOS 8」では実現しない可能性もあるとのことです。',
 'BookLive「東京弁アクセントの変容」の販売促進戦略として、試し読みができるというシステムは完璧なアイデアではないでしょうか。いつでも大雑把なあらすじを読むことが可能なので、ちょっとした時間を活用して漫画を選択することができます。',
 'BookLive恋する絶対の法則の拡販戦略として、試し読みが可能という仕組みは斬新なアイデアだと思われます。',
 'Catalyst Control Center Linuxエディションの初期化に問題がありました。次のことが原因である可能性があります。',
 'Cisco IOS XEソフトウェアはまたこの脆弱性から影響を受けます。',
 'Erlideはemacsの実行可能な代替策になっているのでしょうか?',
 'Have I been pwned?は可能な限り多くのデータを保有していますが、流出の規模が小さかったものについては含まれていない可能性があるためです。',
 'IOCは性転換手術まで認めているようですが、今回の件はどうなるでしょう。経過に注目してみましょ